In [1]:
import os

# # Set the proxy environment variables
# os.environ['http_proxy'] = 'http://127.0.0.1:7890'
# os.environ['https_proxy'] = 'http://127.0.0.1:7890'

In [2]:
import ee
import datetime
import itertools
import sys
import urllib.request

from dfply import *
from glob import glob
from tqdm.auto import tqdm

import geopandas as gpd
import shapely
from shapely.geometry import shape,MultiPoint,Point
import geojson
from geojson import Feature,FeatureCollection


import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import geemap

import subprocess
from subprocess import PIPE

In [3]:
ee.Initialize()

##### Define_Basic_Parameters

In [4]:
# define the year range
year_range = '2020_2022'

# define the gee-asset path for exporting
export_path = 'users/wangjinzhulala/China_built_up/03_sample_extract_img_value'

# create the region_dict
# region_dict = dict(zip(['华东','东北','中南','华北','西北','西南'],
#                        ['huadong','dongbei','zhongnan','huabei','xibei','xinan']))

region_dict = dict(zip(['西南'],
                       ['xinan']))

##### Fetch needed images

In [5]:
# read all asset_df into a list
asset_list = [pd.read_csv(f) for f in  glob(f'../../File_trainsition/gee_asset_china/*.csv')]

In [6]:
# concate all asset_df into one df
asset_df = pd.concat(asset_list)
asset_df = asset_df.sort_values(['region','type','year']) >> mask(X.year == year_range)
asset_df

,year,type,path,region
0,2020_2022,Fourier,projects/earthengine-legacy/assets/users/cnbui...,dongbei
4,2020_2022,Landsat_cloud_free,projects/earthengine-legacy/assets/users/cnbui...,dongbei
8,2020_2022,NDVI_NDBI_EVI,projects/earthengine-legacy/assets/users/cnbui...,dongbei
1,2020_2022,Fourier,projects/earthengine-legacy/assets/users/cnbui...,huabei
5,2020_2022,Landsat_cloud_free,projects/earthengine-legacy/assets/users/cnbui...,huabei
9,2020_2022,NDVI_NDBI_EVI,projects/earthengine-legacy/assets/users/cnbui...,huabei
2,2020_2022,Fourier,projects/earthengine-legacy/assets/users/cnbui...,huadong
6,2020_2022,Landsat_cloud_free,projects/earthengine-legacy/assets/users/cnbui...,huadong
10,2020_2022,NDVI_NDBI_EVI,projects/earthengine-legacy/assets/users/cnbui...,huadong
12,2020_2022,Fourier,projects/earthengine-legacy/assets/users/jinzh...,xibei


##### Prepare img_stack for sample_points extractioin

In [7]:
def stack_imgs(asset_df,region_en):
    
    # import Landsat img
    Landsat_df = asset_df >> mask(X.type=='Landsat_cloud_free',X.region==region_en)
    Landsat_img = ee.Image(Landsat_df['path'].tolist()[0])

    # import Fourier img     
    Fourier_df = asset_df >> mask(X.type=='Fourier',X.region==region_en)
    Fourier_img = ee.Image(Fourier_df['path'].tolist()[0])

    # import Fourier img    
    Normalized_df = asset_df >> mask(X.type=='NDVI_NDBI_EVI',X.region==region_en)
    Normalized_img = ee.Image(Normalized_df['path'].tolist()[0])

    # prepare the climate data
    # because the climate data ends in 2019, we will use the 2019 data if requring future data than 2019
    year = '2017_2019' if int(year_range[:4]) > 2019 else year_range
    Climate_mean = ee.Image(f"users/China_bulit/01_Data_preparation/Climate_Mean_{year}")

    # Import DEM/SLOPE Img
    DEM   = ee.Image("USGS/SRTMGL1_003").rename('DEM')
    SLOPE = ee.Terrain.slope(DEM).rename('SLOPE')

    # stack all Imput_Img together
    Stack_img = ee.Image([Landsat_img,
                           Fourier_img,
                           Normalized_img,
                           Climate_mean,
                           DEM,
                           SLOPE])
    
    return Stack_img

##### Img value extraction and export

In [8]:
# function to get tranining samples
def get_sample(region_en,non_built_pt_path = f'''../../Points_Check/
                                                Sample_points_check_tutorial/
                                                04_Checked_pts/
                                                Samples_checked_Merge_2020_2022
                                                '''):
    # read local sample points
    shp_path = f'{non_built_pt_path}/merge_pts_{region_en}.shp'.replace(' ', '').replace('\n', '')
    in_df = gpd.read_file(shp_path)
    
    # gdf to ee.Fe_col
    sample_ee_fe_col = ee.FeatureCollection(FeatureCollection([Feature(geometry=row['geometry'],properties={'Built':row['Built']}) 
                                                           for _,row in in_df.iterrows()]))
    
    return sample_ee_fe_col

In [18]:
# function to export the img_val_extraction
def export_extraction(img,region_en,year_range):
    
    # get name for exporting
    name   = f'Control_sample_ext_img_{region_en}'
    sample = get_sample(region_en)
    
    # extract img value to sample pts
    Ext_sample = img.sampleRegions( collection = sample, 
                                    properties = ['Built'], 
                                    scale      = 30, 
                                    geometries = True)
    
    # set band_order to none, to avoid error
    Ext_sample = Ext_sample.set({'band_order':'none'})

    # exporting to gee aeest
    task = ee.batch.Export.table.toAsset(   collection  = Ext_sample,
                                            description = f'{name}_{year_range}',
                                            assetId     = f'{export_path}/{name}_{year_range}')
    task.start()

#     # download to local drive
#     url = Ext_sample.getDownloadURL('csv')
#     urllib.request.urlretrieve(url, f'./Data/{name}_{year_range}.csv')

    # print out the process
    print(f'{name}_{year_range}')

In [19]:
# finally, do the export
for region_cn, region_en in tqdm(region_dict.items(),total=len(region_dict)):
    #1) img for extraction
    stack_img = stack_imgs(asset_df,region_en)
    #2) export 
    export_extraction(stack_img,region_en,year_range)

  0%|          | 0/1 [00:00<?, ?it/s]

Control_sample_ext_img_xinan_2020_2022
